# Toxicity Detection Model

Get live data with `get_comments.py` and transform Kaggle data set with `transform_train.py`

Get libraries by running `pip install -r requirements.txt`

This is a deep learning model.

In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

from wordcloud import STOPWORDS, wordcloud
import re
from fuzzywuzzy import fuzz, process

from tqdm import tqdm

# Load Dataset

In [ ]:
df_train = pd.read_csv('./data/new_train.csv')

### Data Cleansing
<ul>
    <li>Removal of special characters</li>
    <li>Expanding contractions</li>
    <li>Lowering text</li>
    <li>Replacing Obfuscated Profane Words</li>
</ul>

In [ ]:
def decontracted(phrase):
    '''
    This function decontracts words like won't to will not
    '''

    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase

In [ ]:
def removeNonPrintable(com):
    com = com.replace('\\r', ' ')
    com = com.replace('\\n', ' ')
    com = com.replace('\\t', ' ')
    com = com.replace('\\"', ' ')
    return com

In [ ]:
def getUniqueWords(comments):
    unique_words = set()
    for comment in tqdm(comments):
        words = comment.split(" ")
        for word in words:
            if len(word) > 2:
                unique_words.add(word)
    
    return unique_words

In [ ]:
def getProfaneWords():
    profane_words = []
    with open("./data/bad-words.txt","r") as f:
        for word in f:
            word = word.replace("\n","")
            profane_words.append(word)
    return profane_words
    

In [ ]:
def createMappingDict(profane_words, unique_words):
    # mapping dictionary
    mapping_dict = dict()
    
    # looping through each profane word
    for profane in tqdm(profane_words):
        mapped_words = set()
        
        # looping through each word in vocab
        for word in unique_words:
            # mapping only if ratio > 80
            try:
                if fuzz.ratio(profane,word) > 80:
                    mapped_words.add(word)
            except:
                pass
                
        # list of all vocab words for given profane word
        mapping_dict[profane] = mapped_words
    
    return mapping_dict

In [ ]:
def replaceWords(corpus, mapping_dict):
    processed_corpus = []

    for document in tqdm(corpus):

        # words = document.split()

        for mapped_word, v in mapping_dict.items():
            
            document = re.sub(r'\{word}\b'.format(word = v), mapped_word, document)

            # for target_word in v:

            #     for i, word in enumerate(words):
            #         if word == target_word:
            #             words[i] = mapped_word

        # document = " ".join(words)
        document = document.strip()

        processed_corpus.append(document)

    return processed_corpus



In [ ]:
def final_processing(corpus):
    '''
    Function applies final processing steps post profane mapping such as removing special characters,
    punctuations etc.
    '''
    processed_comments = []
    processed_words = []
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
    "]+", re.UNICODE)


    # looping through each comment in corpus
    for comment in tqdm(corpus):
        comment = re.sub(emoj, '', comment)
        comment = re.sub(r'http\S+', '', comment)
        comment = re.sub("\B\#\w+", '', comment)
        comment = re.sub("\B\@\w+", '', comment)
        comment = re.sub('[^A-Za-z\s]+',"", comment) # retain only letters
        for word in comment.split():
            if len(word) >= 3:
                processed_words.append(word)
            
        comment = " ".join(processed_words)
        
        processed_comments.append(comment.strip())
    
    return processed_comments

In [ ]:
def cleanComments(comments):
    processed_comments = []
    for comment in comments:
        comment = decontracted(comment)
        comment = removeNonPrintable(comment)

        # Lower comment
        processed_comments.append(comment.lower().strip())
    
    profane_words = getProfaneWords()
    unique_words = getUniqueWords(processed_comments)
    profane_dict = createMappingDict(profane_words, unique_words)
    processed_comments = replaceWords(comments, profane_dict)
    return final_processing(processed_comments)

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
fig, axis = plt.subplots(len(df_train.columns)-1, 1, figsize = (12,30))
for i, col in enumerate(df_train.columns[1:]):
    counts, bins = np.histogram(df_train[col].values, bins = 30)
    axis[i].hist(bins[:-1], bins, weights=counts)
    axis[i].title.set_text(col)
plt.show()

In [ ]:
comments_lens = df_train.Comment.str.len()
counts, bins = np.histogram(comments_lens.values, bins = 50)
plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
for i in range(11):
    print(f'{i*10}th Percentile Value = {np.percentile(comments_lens, i*10)}')

In [ ]:
for i in range(11):
    print(f'{90+i}th Percentile Value = {np.percentile(comments_lens, 90 + i)}')

In [ ]:
processed_comments = cleanComments(df_train.Comment.values[:500])

# Model Building

Turn dataset into list(list of tokens, scores x6))

In [ ]:
detections = df_train.loc[:,"Toxicity": "Threat"].__array__()[:500]
mapped = np.array([[processed_comments[i], detections[i]] for i in list(range(len(processed_comments)))], dtype=object)

In [ ]:
np.random.shuffle(mapped)

In [ ]:
train_dataset = mapped[: int(mapped.shape[0] * .9) ]
test_dataset = mapped[int(mapped.shape[0] * .9) :]

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
VOCAB_SIZE = 1000
encoder = layers.TextVectorization(max_tokens = VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
model = keras.Sequential([
	encoder,
	layers.Embedding(
		input_dim = len(encoder.get_vocabulary()),
		output_dim = 64,
		# Use masking to handle the variable sequence lengths
		mask_zero = True
	),
	layers.Dropout(0.8),
	layers.Bidirectional(layers.LSTM(64)),
	layers.Dropout(0.6),
	layers.Dense(32, activation='relu'),
	layers.Dropout(0.5), # ?
	layers.Dense(6)
])

model.compile(loss = keras.losses.BinaryCrossentropy(from_logits = True),
			  optimizer = keras.optimizers.adam_v2.Adam(1e-4),
			  metrics = ['Toxicity', 'Severe_Toxicity', 'Identity_Attack', 'Insult', 'Profanity', 'Threat'])

In [ ]:
history = model.fit(train_dataset,
					epochs = 20,
					steps_per_epoch = 50,
					validation_data = test_dataset,
					validation_steps = 30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)